In [ ]:
## Installing the necessary libraries and environments here
%pip install pandas scikit-learn seaborn statsmodels

#### 1. Transform the dataset
- Loading the dataset and inspecting

In [3]:
import pandas as pd

# Load the dataset
file_path = r"D:\Data Mining\Data Mining Exam\Part A\Bike_Sales.xlsx"
df = pd.read_excel(file_path)

#check the structure before transformation
df.info() # 113036 rows, 18 columns

#Display first 5 rows
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113036 entries, 0 to 113035
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              113036 non-null  datetime64[ns]
 1   Day               113036 non-null  int64         
 2   Month             113036 non-null  object        
 3   Year              113036 non-null  int64         
 4   Customer_Age      113036 non-null  int64         
 5   Age_Group         113036 non-null  object        
 6   Customer_Gender   113036 non-null  object        
 7   Country           113036 non-null  object        
 8   State             113036 non-null  object        
 9   Product_Category  113036 non-null  object        
 10  Sub_Category      113036 non-null  object        
 11  Product           113036 non-null  object        
 12  Order_Quantity    113036 non-null  int64         
 13  Unit_Cost         113036 non-null  int64         
 14  Unit

,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,2017-01-01,1,January,2017,17,Youth (<25),M,Canada,British Columbia,Bikes,Road Bikes,"Road-250 Red, 44",2,1519,2443,1848,3038,4886
1,2017-01-01,1,January,2017,23,Youth (<25),M,Australia,Victoria,Bikes,Mountain Bikes,"Mountain-200 Black, 46",2,1252,2295,2086,2504,4590
2,2017-01-01,1,January,2017,33,Young Adults (25-34),F,France,Yveline,Bikes,Road Bikes,"Road-150 Red, 48",2,2171,3578,2814,4342,7156
3,2017-01-01,1,January,2017,39,Adults (35-64),M,United States,Washington,Bikes,Road Bikes,"Road-550-W Yellow, 38",2,713,1120,814,1426,2240
4,2017-01-01,1,January,2017,42,Adults (35-64),M,United States,California,Bikes,Road Bikes,"Road-750 Black, 44",2,344,540,392,688,1080


#### Transforming the dataset "Bike_Sales.xlsx"

In [5]:
#Removing rows with missing values and checking for duplicates

# Check for missing values
missing_rows = df.isnull().any(axis=1).sum()
print(f"Number of rows with missing values: {missing_rows}")

# Drop rows with missing values if any exist
if missing_rows > 0:
    df = df.dropna()
    print("Rows with missing values have been removed.")
else:
    print("No missing values found.")
    
#check number of rows duplicated
"""
Duplicates will not be deleted because these are sales transactions with no unique identifiers
for each sale transaction, duplication is expected and still meaningful in this context"
"""
print("Number of duplicated rows:", df.duplicated().sum()) 

# data after cleaning missing values
df.head()

Number of rows with missing values: 0
No missing values found.
Number of duplicated rows: 1007


,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,2017-01-01,1,January,2017,17,Youth (<25),M,Canada,British Columbia,Bikes,Road Bikes,"Road-250 Red, 44",2,1519,2443,1848,3038,4886
1,2017-01-01,1,January,2017,23,Youth (<25),M,Australia,Victoria,Bikes,Mountain Bikes,"Mountain-200 Black, 46",2,1252,2295,2086,2504,4590
2,2017-01-01,1,January,2017,33,Young Adults (25-34),F,France,Yveline,Bikes,Road Bikes,"Road-150 Red, 48",2,2171,3578,2814,4342,7156
3,2017-01-01,1,January,2017,39,Adults (35-64),M,United States,Washington,Bikes,Road Bikes,"Road-550-W Yellow, 38",2,713,1120,814,1426,2240
4,2017-01-01,1,January,2017,42,Adults (35-64),M,United States,California,Bikes,Road Bikes,"Road-750 Black, 44",2,344,540,392,688,1080


In [ ]:
# Finding outliers and eliminating them using IQR method

# Function to detect and remove outliers
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remove outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

# Columns to check for outliers
num_columns = ["Customer_Age", "Unit_Cost", "Unit_Price", "Profit", "Cost", "Revenue"]

# Removing outliers
df_cleaned = remove_outliers(df, num_columns)

# new dataset after removing outliers
#df_cleaned.head()

#### Generating a machine learning model to classify the variable "Age_Group" in the transformed dataset
The following observations can be made about the 18 colums of the dataset
- Temporal variable include: Date, Day, Month, Year
- Continous variables include: Customer_Age, Unit_Cost, Unit_Price, Profit, Cost, Revenue
- Nominal(categorical) variables include: Age_Group, Customer_Gender, Country, State, Product_Category, Sub_Category, Product
- Dicrete variable: Order_Quantity 

In [9]:
#Converting Data Types

# Convert and formating date
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date']).dt.strftime('%Y-%m-%d')

# converting categoricals
categorical_cols = ['Age_Group', 'Customer_Gender', 'Country', 'State', 'Product_Category', 'Sub_Category', 'Product']
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].astype('category')

# Converting continous variables
continuous_cols = ['Customer_Age', 'Unit_Cost', 'Unit_Price', 'Profit', 'Cost', 'Revenue']
df_cleaned[continuous_cols] = df_cleaned[continuous_cols].astype(float)

# Check data types
#df_cleaned.info()

In [10]:
# Saving the cleaned and transformed excel file
transformed_file_path = r"D:\Data Mining\Data Mining Exam\Part A\Nantume.xlsx"
df_cleaned.to_excel(transformed_file_path, index=False)

In [11]:
# Training a Classification Model for "Age_Group"

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Loading the transformed dataset
transformed_file_path = r"D:\Data Mining\Data Mining Exam\Part A\Nantume.xlsx"
df_new = pd.read_excel(transformed_file_path)

# Encoding categorical variables
categorical_columns = ["Age_Group", "Customer_Gender", "Country", "State", "Product_Category", "Sub_Category", "Product"]
encoders = {col: LabelEncoder() for col in categorical_columns}
for col in categorical_columns:
    df_new[col] = encoders[col].fit_transform(df_new[col])

##### Determining First, which features (columns) are correlated to "Age_Group"

In [12]:
# Determining the best features correlated to "Age_Group" (categorical) for the model

import numpy as np
from scipy.stats import f_oneway, chi2_contingency

# Numerical and categorical features
numerical_features = ["Customer_Age", "Order_Quantity", "Unit_Cost", "Unit_Price", "Profit", "Cost", "Revenue"]
categorical_features = ["Customer_Gender", "Country", "State", "Product_Category", "Sub_Category", "Product"]

# Target variable
y = df_new["Age_Group"]

# statistical significance threshold
alpha = 0.05  

# ANOVA Test for the Numerical Features
anova_correlated = []
anova_not_correlated = []

for feature in numerical_features:
    groups = [df_new[df_new["Age_Group"] == cat][feature] for cat in df_new["Age_Group"].unique()]
    stat, p_value = f_oneway(*groups)
    if p_value < alpha:
        anova_correlated.append(feature)
    else:
        anova_not_correlated.append(feature)

# Chi-Square Test for the Categorical Features
chi2_correlated = []
chi2_not_correlated = []

for feature in categorical_features:
    contingency_table = pd.crosstab(df_new[feature], y)
    stat, p_value, _, _ = chi2_contingency(contingency_table)
    if p_value < alpha:
        chi2_correlated.append(feature)
    else:
        chi2_not_correlated.append(feature)

# Displaying Results
print("Features Correlated with 'Age_Group' (p-value < 0.05)")
print("- Numerical:", anova_correlated)
print("- Categorical:", chi2_correlated)

print("\nFeatures NOT Correlated with 'Age_Group' (p-value >= 0.05)")
print("- Numerical:", anova_not_correlated)
print("- Categorical:", chi2_not_correlated)

Features Correlated with 'Age_Group' (p-value < 0.05)
- Numerical: ['Customer_Age', 'Unit_Cost', 'Unit_Price', 'Profit', 'Cost', 'Revenue']
- Categorical: ['Customer_Gender', 'Country', 'State', 'Product_Category', 'Sub_Category', 'Product']

Features NOT Correlated with 'Age_Group' (p-value >= 0.05)
- Numerical: ['Order_Quantity']
- Categorical: []


In [13]:
# Generating the Classification model for "Age_Group"
import joblib

# Features below are determined by the correlation tests above, only correlated features are considered
feature_cols = ["Customer_Age", "Customer_Gender", "Country", "State", "Product_Category", 
                "Sub_Category", "Product", "Unit_Cost", "Unit_Price", "Profit", "Revenue", "Cost"]

X_class = df_new[feature_cols]
y_class = df_new["Age_Group"]

# Splitting data
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Training a RandomForest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_class, y_train_class)

# Saving the classification model
joblib.dump(clf, r"D:\Data Mining\Data Mining Exam\Part A\age_predictor.joblib")

['D:\\Data Mining\\Data Mining Exam\\Part A\\age_predictor.joblib']

##### Generating a Prediction Model for "Revenue"
- 1. Feature selection
- 2. Generate model
- 3. Predict Revenue

In [14]:
# 1. Selecting best features for predicting "Revenue"

# Correlation matrix (for numerical variables)
numericals = ["Customer_Age", "Order_Quantity", "Unit_Cost", "Unit_Price", "Profit", "Cost", "Revenue"]

correlation_matrix = df_new[numericals].corr()

# Correlation values for Revenue (for numericals)
print("Correlation Matrix:\n")
print(correlation_matrix["Revenue"].sort_values(ascending=False))

# ANOVA tests (for categoricals)
from scipy.stats import f_oneway

anova_results = {}

for feature in categorical_columns:
    groups = [df_new[df_new[feature] == cat]["Revenue"] for cat in df_new[feature].unique()]
    stat, p_value = f_oneway(*groups)
    anova_results[feature] = p_value

# Sort features by p-value (ascending)
sorted_anova = sorted(anova_results.items(), key=lambda x: x[1])

# Print sorted results with formatting
print("\nANOVA Test for Categorical Features:\n")
for feature, p_value in sorted_anova:
    correlation_status = "Correlated (significant difference)" if p_value < 0.05 else "Not Correlated (no significant difference)"
    
    # Format very small p-values
    p_str = "< 0.00001" if p_value < 1e-5 else f"{p_value:.5f}"
    
    print(f"{feature:<20}: {correlation_status} \t(p = {p_str})")


Correlation Matrix:

Revenue           1.000000
Profit            0.972623
Cost              0.934458
Unit_Price        0.604248
Unit_Cost         0.468021
Order_Quantity    0.284138
Customer_Age      0.034189
Name: Revenue, dtype: float64

ANOVA Test for Categorical Features:

Sub_Category        : Correlated (significant difference) 	(p = < 0.00001)
Product             : Correlated (significant difference) 	(p = < 0.00001)
Product_Category    : Correlated (significant difference) 	(p = < 0.00001)
State               : Correlated (significant difference) 	(p = < 0.00001)
Age_Group           : Correlated (significant difference) 	(p = < 0.00001)
Country             : Correlated (significant difference) 	(p = < 0.00001)
Customer_Gender     : Correlated (significant difference) 	(p = < 0.00001)


In [15]:
# importing the necessary libraries

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score

In [17]:
# Loading the training data
df_train = df_new.copy()

# Splitting features and target variables 
# features have been determined by the results from the correlation tests above
feature_columns = ['Customer_Age', 'Age_Group', 'Customer_Gender', 'Country', 'State', 'Product_Category', 'Sub_Category', 'Product',
                    'Order_Quantity', 'Unit_Cost', 'Unit_Price', 'Cost']  # 'Profit is eliminated (Profit is derived from Cost and Revenue)

X = df_train[feature_columns]  # Features
y = df_train["Revenue"]  # Target variable

#Splitting data into training and testing sets (for cross-validation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluating model
y_pred = model.predict(X_test)
print("Model Performance:")
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"R² Score: {r2_score(y_test, y_pred)}")


Model Performance:
MAE: 0.0001772078951159822
R² Score: 0.9999999916975825


##### The Mean Absolute Error (MAE) of the model is very small: 0.00018 (close to zero), indicating that the model's predictions are almost accurate and very close to the actual values. The model is making minimal errors in its predictions. The R² score being extremely close to 1 (is 0.999) further supports the model's effectiveness, as it implies that the model explains almost all the variability in the target variable (Revenue) based on the selected features.

In [18]:
# Cross-validation to evaluate the model's performance across different subsets of the data.
from sklearn.model_selection import cross_val_score

# Performing 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores:", -cv_scores)
print("Average Cross-Validation MAE:", -cv_scores.mean())

Cross-Validation MAE Scores: [1.14824953e-03 7.77985881e-05 4.06310785e-04 1.85865572e-04
 5.28059938e-04]
Average Cross-Validation MAE: 0.00046925688281650873


##### The average MAE across the 5 folds of the data is 0.000469, which is extremely small. This means that, on average, the model's predictions are very close to the actual values, with only a tiny error.

In [19]:
import joblib

# Saving the model
joblib.dump(model, "Revenue_predictor.joblib")
print("Model saved as 'Revenue_predictor.joblib")

Model saved as 'Revenue_predictor.joblib


#### Predicting the "Revenue" in the testing dataset "Bike_sales_Uganda.xlsx"

In [20]:
# Loading test data
df_test = pd.read_excel("Bike_sales_Uganda.xlsx")

# Select only the same feature columns as df_train
df_test = df_test[feature_columns]

# Loading model
revenue_predictor = joblib.load("Revenue_predictor.joblib")

# Initialize the LabelEncoder and store the original encodings
label_encoders = {} 

# Encode categorical columns in df_test the same way as df_train
categorical_columns = ['Age_Group', 'Customer_Gender', 'Country', 'State', 'Product_Category', 'Sub_Category', 'Product']
for col in categorical_columns:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col])  # Encode the column
    label_encoders[col] = le  # Save the encoder for later decoding

# Predict
df_test["Predicted_Revenue"] = revenue_predictor.predict(df_test)

# Decode categorical columns back to their original values
for col in categorical_columns:
    df_test[col] = label_encoders[col].inverse_transform(df_test[col])

# Saving predictions
df_test.to_csv("Revenue_prediction.csv", index=False)

print("Predictions saved as 'Revenue_prediction.csv")

Predictions saved as 'Revenue_prediction.csv
